<a href="https://colab.research.google.com/github/juan19932carlos/Notebooks/blob/master/misc/gmail-api-uber-travels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pandas as pd
import re
import numpy as np

## Get user credentials

In [ ]:
!ls drive/MyDrive/*.json

drive/MyDrive/client_secret_313892779657-02gs93mmee20h8v0nl191cgihf11cfhr.apps.googleusercontent.com.json
drive/MyDrive/oauth-juan-token.json


In [10]:
creds = None
creds_path = './drive/MyDrive/oauth-juan-token.json'
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

if os.path.exists(creds_path):
  creds = Credentials.from_authorized_user_file(creds_path, SCOPES)
else :
  raise Exception("No credentials found")

if creds and creds.expired and creds.refresh_token:
  creds.refresh(Request())

# Save the credentials for the next run
with open(creds_path, "w") as token:
  token.write(creds.to_json())

creds.scopes

['https://www.googleapis.com/auth/gmail.readonly']

In [33]:
gmailService = build("gmail", "v1", credentials=creds)
msgResulst = gmailService.users().messages().list(userId='me', q='Recibos de Uber', maxResults=500).execute()
msgResulst['resultSizeEstimate']

212

In [36]:
msgs_df = pd.DataFrame(msgResulst['messages'])
msgs_df.describe()

,id,threadId
count,212,212
unique,212,145
top,1902ea97d9bdf863,188933e24063e744
freq,1,5


In [49]:
msgs_df['snippet'] = msgs_df['id'].apply(lambda x: gmailService.users().messages().get(userId='me', id=x).execute()['snippet'] )
msgs_df.head()

,id,threadId,snippet
0,1902ea97d9bdf863,1902da66cdc817cd,Total 22.050 COP 26 de mayo de 2024 Gracias po...
1,1902da66cdc817cd,1902da66cdc817cd,Total 12.060 COP 18 de junio de 2024 Gracias p...
2,19024bc14fa2fc01,19024bc14fa2fc01,Total 9.940 COP 17 de junio de 2024 Gracias po...
3,190242967d2637f4,190242967d2637f4,Total 10.930 COP 16 de junio de 2024 Gracias p...
4,1902278103dd1401,1902278103dd1401,Total 17.780 COP 16 de junio de 2024 Gracias p...


In [80]:
regex = re.compile(r"Total ((\d+[\.,]?)+) ([A-Z]+).+(\d{1,2} de \w+ de \d{4})")

msgs_df['re_result'] = msgs_df['snippet'].apply(regex.findall)
mask = msgs_df['re_result'].apply(lambda x: len(x) > 0)
msgs_df[~mask]

,id,threadId,snippet,re_result
81,18c571bd6bfe2369,18c571bd6bfe2369,Se te cobrará solo por el viaje. Más informaci...,[]
92,18bb446b90c0449a,18bb446b90c0449a,Se te cobrará solo por el viaje. Más informaci...,[]
131,18a2d7e7b3c6c017,18a2d7e7b3c6c017,"Cargo por cancelación 5.430,00 COP 25 de agost...",[]
200,18826386c7954c15,18823fd31d8754c6,Cargo por cancelación 1.100 CLP 16 de mayo de ...,[]


In [89]:
travel_cost_df = msgs_df[mask]['re_result'].apply(lambda x: pd.Series(x[0])[[0, 2, 3]])
travel_cost_df.columns = ['amount', 'currency', 'date']
travel_cost_df.head()

,amount,currency,date
0,22.050,COP,6 de mayo de 2024
1,12.060,COP,8 de junio de 2024
2,9.940,COP,7 de junio de 2024
3,10.930,COP,6 de junio de 2024
4,17.780,COP,6 de junio de 2024


In [95]:
# prompt: filter COP

cop_travels = travel_cost_df[travel_cost_df['currency'] == 'COP']
cop_travels.describe()


,amount,currency,date
count,196,196,196
unique,174,1,85
top,"23.200,00",COP,6 de agosto de 2023
freq,3,196,8


In [96]:
cop_travels['amount_number'] = cop_travels['amount'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
cop_travels.head()

<ipython-input-96-c70b141ade94>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cop_travels['amount_number'] = cop_travels['amount'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))


,amount,currency,date,amount_number
0,22.050,COP,6 de mayo de 2024,22050.0
1,12.060,COP,8 de junio de 2024,12060.0
2,9.940,COP,7 de junio de 2024,9940.0
3,10.930,COP,6 de junio de 2024,10930.0
4,17.780,COP,6 de junio de 2024,17780.0


In [97]:
cop_travels['amount_number'].sum()

3171360.0

In [100]:
display(3171360.0/196)
display(360/196)

16180.408163265307

1.836734693877551